Calibration for Powder Diffraction
==================
There are two general interfaces used for calibrating instruments for powder diffraction that are covered in this tutorial
* [Calibration Information](#Calibration-Information)
* [CalibrateRectangularDetectors](#CalibrateRectangularDetectors)
* [PDCalibration](#PDCalibration)

The tutorial also covers information about masking calibration files

In [ ]:
# setup plotting
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
plt.xkcd() # comment this out for serious science
# useful modules
import os
# ...and, of course, mantid
from mantid.simpleapi import *

Calibration Information
--------------------------
For all of the calibrations there needs to be a "known" lattice constant to calibrate against when determining the tof to dspacing conversion factors. Traditionally at SNS diamond is used for this. Mantid has a facility for [crystal structure and reflections](http://docs.mantidproject.org/nightly/concepts/CrystalStructureAndReflections.html) built in.

In [ ]:
from mantid.geometry import CrystalStructure, ReflectionGenerator
latticeConstants = ['3.56792' for i in xrange(3)] # 
diamond = CrystalStructure(' '.join(latticeConstants),
                           "F d -3 m", "C 0 0 0 1.0 0.05")
generator = ReflectionGenerator(diamond)
dvalues = generator.getDValues(generator.getUniqueHKLs(.3,3.))
print dvalues

This generates way more d-values than what is traditionally used, so instead will use a hard-coded list.

In [ ]:
dvalues = [2.0600,1.2615,1.0758,0.8920,0.8186,0.7283,0.6867,0.6307,0.5947,0.5642,0.5441,
           0.5150,0.4996, 0.4768,0.4645,0.4205,0.3916,0.3499,0.3257,0.3117]

[CalibrateRectangularDetectors](http://docs.mantidproject.org/nightly/algorithms/CalibrateRectangularDetectors-v1.html)
-------------------------------

`CalibrateRectangularDetectors` is a algorithm intended on simplifying a standard workflow for generating calibration files. Since it is relatively mature, it contains properties for some more common ways to calibrate. This tutorial will focus on the `CrossCorrelation=False` option.

In [ ]:
CalibrateRectangularDetectors(RunNumber="PG3_27021", Binning="0.3,-0.0004,3",
                              PeakPositions=dvalues,
                              # control the peak fitting
                              MaxOffset=0.005,
                              CrossCorrelation=False,
                              PeakWindowMax=0.1,
                              # what function to fit with
                              BackgroundType="Flat",
                              PeakFunction="Gaussian",
                              # default grouping used in reduction
                              GroupDetectorsBy="All",
                              # how to save the results
                              SaveAs="calibration",
                              OutputDirectory="/tmp")

The calibration file will be named `/tmp/PG3_calibrate_d27021_YYYY_MM_DD.h5` with `YYYY_MM_DD` filled in with today's date.

[PDCalibration](http://docs.mantidproject.org/nightly/algorithms/PDCalibration-v1.html)
-------------------------------------

In [ ]:
wkspName='NOM_79584'

LoadEventAndCompress(Filename=wkspName, OutputWorkspace=wkspName,
                     MaxChunkSize=16, FilterBadPulses=25)
CropWorkspace(InputWorkspace=wkspName, OutputWorkspace=wkspName, XMin=300, XMax=16666.7)

# generates a workspace of delta-d/d which could be used to estimate uncertainties
#EstimateResolutionDiffraction(InputWorkspace=wkspName, OutputWorkspace='resolution', DeltaTOF=5)

# start from a previous calibration
oldCal='/SNS/NOM/shared/CALIBRATION/2016_2_1B_CAL/NOM_d79330_2016_09_07.h5'

# NOMAD uses tabulated reflections for diamond
dvalues = (0.3117,0.3257,0.3499,0.4205,0.4645,0.4768,0.4996,0.5150,0.5441,0.5642,0.5947,
           0.6307,.6866,.7283,.8185,.8920,1.0758,1.2615,2.0599)

PDCalibration(SignalWorkspace=wkspName,
              TofBinning=[300,-.001,16666.7],
              PreviousCalibration=oldCal,
              PeakPositions=dvalues,
              StartFromObservedPeakCentre=True,
              OutputCalibrationTable='new_cal',
              CalibrationParameters='DIFC')

dbinning=(.01,-.001,3.)
AlignDetectors(InputWorkspace=wkspName, OutputWorkspace=wkspName, CalibrationWorkspace='new_cal')
CropWorkspace(InputWorkspace=wkspName, OutputWorkspace=wkspName,
              XMin=dbinning[0], XMax=dbinning[2])
Rebin(InputWorkspace=wkspName, OutputWorkspace=wkspName, Params=dbinning)

[DetectorDiagnostic](http://docs.mantidproject.org/nightly/algorithms/DetectorDiagnostic-v1.html) uses statistical criteria for determining what pixels should be used to produce final data

In [ ]:
LoadEventAndCompress(Filename='NOM_78771', OutputWorkspace='NOM_78771',
                    MaxChunkSize=8, FilterBadPulses=25)
DetectorDiagnostic(InputWorkspace='NOM_78771', OutputWorkspace='NOM_mask_detdiag',
                   RangeLower=300, RangeUpper=16666.7, # TOF range to use
                   LowThreshold=10, # minimum number of counts for a detector
                   LevelsUp=1) # median calculated from the tube

The result of `DetectorDiagnostic` can be combined with the result of the mask generated by `PDCalibration` using [BinaryOperateMasks](http://docs.mantidproject.org/nightly/algorithms/BinaryOperateMasks-v1.html)

In [ ]:
BinaryOperateMasks(InputWorkspace1='new_cal_mask', InputWorkspace2='NOM_mask_detdiag',
                   OperationType='OR', OutputWorkspace='NOM_mask_final')

The only information missing for `SaveDiffCal` is which pixels to combine to make an output spectrum. This is done using [CreateGroupingWorkspace](http://docs.mantidproject.org/nightly/algorithms/CreateGroupingWorkspace-v1.html). For NOMAD, the `Column` option will generate 6 spectra. An alternative is to generate a grouping file to load with [LoadDetectorsGroupingFile](http://docs.mantidproject.org/nightly/algorithms/LoadDetectorsGroupingFile-v1.html).

In [ ]:
CreateGroupingWorkspace(InstrumentName='NOMAD', GroupDetectorsBy='Group',
                       OutputWorkspace='NOM_group')

In [ ]:
SaveDiffCal(CalibrationWorkspace='new_cal',
             GroupingWorkspace='NOM_group',
             MaskWorkspace='NOM_mask_final',
            Filename='/tmp/NOM_d79584.h5')

[AlignComponents](https://github.com/mantidproject/mantid/blob/master/docs/source/algorithms/AlignComponents-v1.rst)
-----------------
This algorithm can be used to update the instrument geometry itself by optimizing the position of the physical components to the `DIFC` from calibration. From this point down is going to clear out the memory and start over.

In [ ]:
for name in mtd.getObjectNames():
    DeleteWorkspace(Workspace=name)
LoadDiffCal(InstrumentName='NOMAD',
            Filename='/tmp/NOM_d79584.h5',
            WorkspaceName='NOM',
            TofMin=300, TofMax=16667)

First refine the sample position along the beam.

In [ ]:
idf_orig = os.path.join(ConfigService.getInstrumentDirectory(), 'NOMAD_Definition.xml')
AlignComponents(CalibrationTable='NOM_cal',
                MaskWorkspace='NOM_mask',
                InstrumentFilename=idf_orig,
                FitSamplePosition=True,
                Zposition=True)

This creates two workspaces `calWS` which is another representation of the calibration information and `alignedWorkspace` which has the updated instrument. Note that original and final positions are logged at the `Notice` level.

After the sample position is refined, refine the detector positions. The algorithm requires naming the components of the instrument to refine. As an example we'll refine the position and rotation of some of the detector 8-packs. These are called `bank#` in the idf. Starting from where the previous refinement finished is done by giving it `alignedWorkspace` in the subsequent invocations.

In [5]:
components = ['bank%d' % (i) for i in range(47,50)]
components = ','.join(components)
print components
AlignComponents(CalibrationTable='NOM_cal',
                MaskWorkspace='NOM_mask',
                Workspace='alignedWorkspace',
                ComponentList=components,
                Xposition=True, Yposition=True, Zposition=True)

bank47,bank48,bank49


This can be done several times in succession, varying parameters with each call to `AlignComponents`. If the instrument view is up you can watch the changes as they happen.

Finally, you'll want to save the the information out so you can create a new idf. Unfortunately, there isn't a tool that will update the IDF automatically, but `ExportGeometry` will write out the source, sample, and named components to an xml file. You can copy the contents into correct places in the idf.

In [ ]:
ExportGeometry(InputWorkspace='alignedWorkspace',
               Components=components,
               Filename='/tmp/NOMAD_partial_idf.xml')